# 타입B 중간보스

In [1]:
import re
from tqdm.notebook import tnrange
import csv
import json
from os import walk
import os
import pandas as pd
import datetime

In [2]:
# 해당 경로의 파일리스트 추출
def get_filelist(input_repo):
    file_list = []
    for (dirpath, dirnames, filenames) in walk(input_repo):
        file_list.extend(filenames)
        break

    return file_list

In [3]:
# 해당 경로의 csv파일의 경로+파일명 리스트 추출
def get_csvlist(input_repo, file_list):
    csv_list = []
    for i in range(len(file_list)):
        filename, file_extension = os.path.splitext(f'{input_repo}/{file_list[i]}')

        if file_extension == '.csv':
            csv_list.append(filename+file_extension)

    return csv_list

In [4]:
def concat_csvfile(csv_list):
    COLUMNS = ['lineNumber', 'originalText', 'transcription']
    result = pd.concat([pd.read_csv(f) for f in csv_list])    # 라인넘버 기준 안합친파일 합치기
    sorted_result_not_flat = result.sort_values(by=['lineNumber'])    # 작업물 sort by lineNumber
    sorted_result_not_flat.to_csv(f'{output_repo}/temp.csv', index=False, encoding='utf-8-sig')
    sorted_result = pd.read_csv(f'{output_repo}/temp.csv')
    os.remove(f'{output_repo}/temp.csv')
    
    return sorted_result

In [5]:
# 전체 문장에서 태그 안만 뽑음
def save_tag(sentence):
    regex = re.compile('(?<=\<{3}).*(?=\>{3})')
    result = regex.search(sentence)
    
    return result

In [6]:
# 전체 문장으로부터 태그 제거
def remove_tag(sentence):
    result = sentence.replace('<<<', '').replace('>>>', '')
        
    return result

In [7]:
# 태그 내 문장으로부터 법률어휘 추출
def draw_word(sentence_tagremoved):
    regex = re.compile('(?<=\<=).*(?=\>)')
    result = regex.search(sentence_tagremoved)
    
    try:
        return result.group()
    except:
        return 'no word'

In [8]:
# csv형태로 저장
def save_as_csv(input_repo, output_repo, result):
    year = str(datetime.datetime.now().year)[2:]
    month = str(datetime.datetime.now().month)
    day = str(datetime.datetime.now().day)
    hour = str(datetime.datetime.now().hour)
    minute = str(datetime.datetime.now().minute)

    print(f'{output_repo}의 경로에 태그안뽑기_{year}{month}{day}_{hour}{minute}.csv의 이름으로 파일 생성이 완료되었습니다.')

    result.to_csv(f'{output_repo}/태그안뽑기_{year}{month}{day}_{hour}{minute}.csv', index=False, encoding='utf-8-sig')

In [9]:
input_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/concat'
ref_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/accum_concat/법률어휘 살리기 최종'
output_repo = '/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기'

ref_filename = 'total_with_lineNumber.csv'

In [10]:
file_list = get_filelist(input_repo)
csv_list = get_csvlist(input_repo, file_list)
sorted_result = concat_csvfile(csv_list)
ref_word = pd.read_csv(f'{ref_repo}/{ref_filename}')

In [ ]:
result_dataframe = pd.DataFrame(columns=['lineNumber', 'word', 'originalText', 'typeA', 'transcription', 'tag_removed'])
for i in tnrange(len(sorted_result)):
    lineNumber = sorted_result['lineNumber'][i]
    word = list(ref_word[ref_word['lineNumber']==sorted_result['lineNumber'][i]]['word'])[0]
    originalText = list(ref_word[ref_word['lineNumber']==sorted_result['lineNumber'][i]]['org'])[0]
    typeA = sorted_result['originalText'][i]
    transcription = sorted_result['transcription'][i]
    tag_removed = re.sub('(\<=.*?\>{1})','',str(sorted_result['transcription'][i])).replace('<<<', '').replace('>>>', '')
    result_dataframe = result_dataframe.append({'lineNumber':lineNumber, 'word':word, 'originalText':originalText, 'typeA':typeA, 'transcription':transcription, 'tag_removed':tag_removed}, ignore_index=True)

  0%|          | 0/71300 [00:00<?, ?it/s]

In [12]:
sorted_result

,lineNumber,originalText,transcription,name
0,1,"<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이...",NaN
1,2,비록 양 발명의 구성에 <<<서로 다름<=상이>점이 있어도>>> 그 기술분야에서 <...,비록 양 발명의 구성에 <<<서로 다른<=상이> 점이 있어도>>> 그 기술분야에서 ...,NaN
2,3,"[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문...",NaN
3,4,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...,NaN
4,5,"구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...",NaN
...,...,...,...,...
71295,76898,구 대통령선거법 제34조는 '선거운동'고 규정하여 사전선거운동을 금지하고 이를 위반...,구 대통령선거법 제34조는 '선거운동'고 규정하여 사전선거운동을 금지하고 이를 위반...,NaN
71296,76899,****.**.**시행된 **** 선거구의 제16대 국회의원 선거에 <<민사소송을 ...,****.**.**시행된 **** 선거구의 제16대 국회의원 선거에 <<민사소송을 ...,NaN
71297,76900,"<<직전의 재판<=원심>판결 이유에>> 의하면, <<직전의 재판<=원심>은 그>> ...","<<직전의 재판<=원심>판결 이유에>> 의하면, <<직전의 재판<=원심>은 그>> ...",NaN
71298,76901,헌법재판소가 선거운동인지를 판단하는 중요한 기준은 행위의 ‘특정후보자의 당선 내지 ...,헌법재판소가 선거운동인지를 판단하는 중요한 기준은 행위의 ‘특정후보자의 당선 내지 ...,NaN


In [13]:
result_dataframe

,lineNumber,word,originalText,typeA,transcription,tag_removed
0,1,호환,"원심판결 이유에 의하면, 원심은 이 사건 행위가 공정거래법 제3조의2 제1항 제3호...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이...","<<<직전의 재판<=원심>판결 이유에>>> 의하면, <<<직전의 재판<=원심>은 이...","직전의 재판판결 이유에 의하면, 직전의 재판은 이 사건 행위가 공정거래법 제3조의2..."
1,2,호환,비록 양 발명의 구성에 상이점이 있어도 그 기술분야에서 통상의 지식을 가진 자가 보...,비록 양 발명의 구성에 <<<서로 다름<=상이>점이 있어도>>> 그 기술분야에서 <...,비록 양 발명의 구성에 <<<서로 다른<=상이> 점이 있어도>>> 그 기술분야에서 ...,비록 양 발명의 구성에 서로 다른 점이 있어도 그 기술분야에서 일반적인 경우의 지식...
2,3,호환,"[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 호환성이 있어 의장법...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 <<<연결되는 데 문...","[2] 등록의장의 대상물품인 ""스위치""가 독립거래의 대상성 및 연결되는 데 문제가 ..."
3,4,호환,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 호환컴퓨터에 대표적으...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 <<<연결되는 데 문...,③ 이 사건 프로젝터에 사용된 LCD는 ***** 컴퓨터나 그 연결되는 데 문제가 ...
4,5,호환,"구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ...","구 의장법 제2조 제1호에서 말하는 ""물품""이란 독립성이 있는 구체적인 유체동산을 ..."
...,...,...,...,...,...,...
4304,4305,재산인수,"원심은 판시와 같은 이유로, 이 사건 토지의 취득 원인인 약정이 무효인 재산인수에 ...",<<<직전의 재판<=원심>은 어떤 사항에 관하여 판결하여 보임<=판시>와>>> 같은...,<<<직전의 재판<=원심>은 어떤 사항에 관하여 판결한 것과<=판시>>>> 같은 이...,"직전의 재판은 어떤 사항에 관하여 판결한 것과 같은 이유로, 이 사건 토지의 취득 ..."
4305,4306,재산인수,(7) 상법이 재산인수를 회사의 변태설립사항의 하나로 정하여 엄격하게 규제하고 있는...,(7) 상법이 <<<설립 후에 재산을 넘겨받는 계약<=재산인수>를 회사의>>> 변태...,(7) 상법이 <<<설립 후에 재산을 넘겨받는 계약<=재산인수>의 회사의>>> 변태...,(7) 상법이 설립 후에 재산을 넘겨받는 계약의 회사의 변태설립사항의 하나로 정하여...
4306,4307,재산인수,甲이 乙이 장래 설립·운영할 丙 주식회사에 토지를 현물로 출자하거나 매도하기로 약정...,甲이 乙이 장래 설립·운영할 丙 주식회사에 토지를 현물로 <<<사업 진행을 위한 자...,甲이 乙이 장래 설립·운영할 丙 주식회사에 토지를 현물로 <<<사업 진행을 위한 자...,甲이 乙이 장래 설립·운영할 丙 주식회사에 토지를 현물로 사업 진행을 위한 자본을 ...
4307,4308,재산인수,(5) 피고 회사에 이 사건 토지를 양도하기로 한 원고와 원심 공동피고 1 사이의 ...,(5) <<<민사소송에서 소송을 당한 사람<=피고> 회사에>>> 이 사건 토지를 <...,(5) <<<민사소송에서 소송을 당한<=피고> 회사에>>> 이 사건 토지를 <<<권...,(5) 민사소송에서 소송을 당한 회사에 이 사건 토지를 권리나 법률상 지위를 타인에...


In [17]:
typeA

'다만, 선거에 관한 단순한 의견의 <<의견이나 내용을 드러내어 전개함<=개진>·의사의 표시·입후보와>> 선거를 위한 준비행위 또는 <<일반적인 경우<=통상>적인 정당활동은>> 선거운동으로 보지 아니한다고 규정하고 있고, 법 제254조 제3항은 같은 조 제2항에 규정된 방법 외의 방법으로 선거운동기간 전에 선거운동을 한 자에 대한 처벌을 규정하고 있는바, 선거운동을 규율하고 있는 위 각 규정의 내용 및 <<하려는 목적이나 요지<=취지>와 국민의>> 기본권에 관한 과잉제한금지의 원칙을 규정한 헌법 제37조 제2항의 <<하려는 목적이나 요지<=취지> 등에>> 비추어 보면 "선거운동"이라 함은 특정의 선거에 있어서 특정 후보자의 당선 내지 <<표에서 찬성표를 얻음<=득표>를 위한>> 모든 행위 또는 특정 후보자의 <<선거에서 떨어짐<=낙선>을 위한>> 모든 행위로서 당선 또는 <<선거에서 떨어짐<=낙선>을 도모한다는>> 목적의사가 객관적으로 인정될 수 있는 능동적·계획적 행위를 말하며 일상적·의례적·사교적인 행위는 여기에서 제외된다 할 것이고(대법원 ****.**.**선고 92도344 판결, ****.**.**선고 92도1085 판결 참조),'

In [18]:
save_as_csv(input_repo, output_repo, result_dataframe)

/Users/jaytsol/Desktop/Type23 (2)/누적/TypeB태그안뽑기의 경로에 태그안뽑기_211112_022.csv의 이름으로 파일 생성이 완료되었습니다.
